# Getting all the results


The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---
- **<a href="../../QuickStart/usage/#q">Q.()</a>** builds a query that can be used by `run()` or `count()`
- **<a href="../../QuickStart/usage/#qrun">Q.run()</a>** returns data for the specified search 
- **<a href="../../QuickStart/usage/#qcount">Q.count()</a>** returns summary information (counts) data that fit the specified search
- **<a href="../../QuickStart/usage/#columns">columns()</a>** returns entity field names
- **<a href="../../QuickStart/usage/#unique_terms">unique_terms()</a>** returns entity field contents

---
                                                                    
Before we do any work, we need to import several functions from cdapython:
- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents

We're also importing functions from several other packages to make viewing and manipulating tables easier. The `opt.` settings are pre-configuring how itables should display our tables, with scrolling and paging enabled.
Finally, we're telling cdapython to report it's version so we can be sure we're using the one we mean to:

In [1]:
from cdapython import (
    Q, columns, unique_terms, set_default_project_dataset, set_host_url, 
    set_table_version, get_host_url, get_default_project_dataset, get_table_version 
)
import numpy as np
import pandas as pd
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)
import itables.options as opt
opt.maxBytes=0
opt.scrollX="200px"
opt.scrollCollapse=True
opt.paging=True
opt.maxColumns=0

set_default_project_dataset("gdc-bq-sample.dev")
set_host_url("http://35.192.60.10:8080/")
set_table_version("all_merged_subjects_v3_2_final")

print(get_host_url())
print(get_default_project_dataset())
print(get_table_version())
print(Q.get_version())

<IPython.core.display.Javascript object>

http://35.192.60.10:8080/

gdc-bq-sample.dev

all_merged_subjects_v3_2_final

2023.4.18

The CDA indexes tens of thousands of subjects, researchsubjects, speciments, and their diagnosis data. CDA also indexes more than 45 million files. To keep search results from being overwhelming, CDA limits search results to the first 100 records by default:

In [2]:
myquery = Q('primary_diagnosis_site = "brain"')
brainresults = myquery.subject.run()
brainresults

Getting results from database

Total execution time: 0
                            min 3.888 sec 3888 ms


            
            Offset: 0
            Count: 100
            Total Row Count: 2438
            More pages: True
            


---

- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows are in the full results table
- **More pages:** This is always a True or False. False means that our current page has all the available results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---

This preview behaviour is great for search, but not when you are ready to download the information. The `get_all` feature is how you retrieve all the results for your final query and not just the first 100. 

You can have your full results output to a dataframe or a list.

## Results to a dataframe



In [3]:
all_brain_results = brainresults.get_all().to_dataframe()

In [4]:
all_brain_results # view the dataframe

subject_id  \
0                                  GENIE.GENIE-NKI-8IN3   
1                                     TCGA.TCGA-12-1601   
2     CPTAC3 Discovery and Confirmatory.GTEX-Q2AG-00...   
3                    CPTAC.GTEX-NPJ7-0011-R10A-SM-HAKXW   
4                                  GENIE.GENIE-NKI-U4QV   
...                                                 ...   
2433                          GENIE.GENIE-GRCC-18fce8dd   
2434                          GENIE.GENIE-GRCC-774ad24a   
2435                          GENIE.GENIE-GRCC-22355f27   
2436                          GENIE.GENIE-GRCC-4ac049a8   
2437                          GENIE.GENIE-GRCC-99a7f076   

                                     subject_identifier       species  \
0     [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
1     [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
2     [{'system': 'PDC', 'field_name': 'Case.case_id...  Homo sapiens   
3     [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
4     [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
...                                                 ...           ...   
2433  [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
2434  [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
2435  [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
2436  [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
2437  [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   

               sex                    race               ethnicity  \
0             male                 Unknown                 Unknown   
1     not reported            not reported            not reported   
2           Female                   White                    None   
3           female                   white            not reported   
4           female                 Unknown                 Unknown   
...            ...                     ...                     ...   
2433        female  not allowed to collect  not allowed to collect   
2434        female  not allowed to collect  not allowed to collect   
2435          male  not allowed to collect  not allowed to collect   
2436          male  not allowed to collect  not allowed to collect   
2437          male  not allowed to collect  not allowed to collect   

      days_to_birth                  subject_associated_project  vital_status  \
0          -24837.0                                 [GENIE-NKI]  Not Reported   
1               NaN                                  [TCGA-GBM]  Not Reported   
2               NaN           [PDC000204, PDC000205, PDC000245]          Dead   
3               NaN  [CPTAC-3, PDC000204, PDC000205, PDC000245]          Dead   
4          -27759.0                                 [GENIE-NKI]  Not Reported   
...             ...                                         ...           ...   
2433       -26663.0                                [GENIE-GRCC]  Not Reported   
2434       -11688.0                                [GENIE-GRCC]  Not Reported   
2435       -23741.0                                [GENIE-GRCC]  Not Reported   
2436       -20454.0                                [GENIE-GRCC]  Not Reported   
2437       -17897.0                                [GENIE-GRCC]  Not Reported   

      days_to_death      cause_of_death  
0               NaN                None  
1               NaN                None  
2               NaN  Not Cancer Related  
3               0.0                None  
4               NaN                None  
...             ...                 ...  
2433            NaN                None  
2434            NaN                None  
2435            NaN                None  
2436            NaN                None  
2437            NaN                None  

[2438 rows x 11 columns]

## Results to a list

In [5]:
list_all_brain_results = brainresults.get_all().to_list()
list_all_brain_results

[{'subject_id': 'GENIE.GENIE-NKI-8IN3',
  'subject_identifier': [{'system': 'GDC',
    'field_name': 'case.submitter_id',
    'value': 'GENIE-NKI-8IN3'}],
  'species': 'Homo sapiens',
  'sex': 'male',
  'race': 'Unknown',
  'ethnicity': 'Unknown',
  'days_to_birth': -24837,
  'subject_associated_project': ['GENIE-NKI'],
  'vital_status': 'Not Reported',
  'days_to_death': None,
  'cause_of_death': None},
 {'subject_id': 'TCGA.TCGA-12-1601',
  'subject_identifier': [{'system': 'GDC',
    'field_name': 'case.submitter_id',
    'value': 'TCGA-12-1601'}],
  'species': 'Homo sapiens',
  'sex': 'not reported',
  'race': 'not reported',
  'ethnicity': 'not reported',
  'days_to_birth': None,
  'subject_associated_project': ['TCGA-GBM'],
  'vital_status': 'Not Reported',
  'days_to_death': None,
  'cause_of_death': None},
 {'subject_id': 'CPTAC3 Discovery and Confirmatory.GTEX-Q2AG-0011-R10A-SM-HAKXT',
  'subject_identifier': [{'system': 'PDC',
    'field_name': 'Case.case_id',
    'value': '1

# Advanced usage

If you'd like more control over your output, you can use the `paginator`. It will also get all the data, but requires you to write the looping code. By using the paginator and a combination of limits, offsets, and page sizes, you can download all or part of the dataset, at any rate.

In the simplest case, you create an empty dataframe for the data to land in, then use the paginator in a loop to get all the results:

In [6]:
mydf = pd.DataFrame()
for i in myquery.subject.run().paginator(to_df=True):
    mydf = pd.concat([mydf, i])

Getting results from database

Total execution time: 0
                            min 3.478 sec 3478 ms

In [7]:
mydf  # view the dataframe

subject_id  \
0                                GENIE.GENIE-NKI-8IN3   
1                                   TCGA.TCGA-12-1601   
2   CPTAC3 Discovery and Confirmatory.GTEX-Q2AG-00...   
3                  CPTAC.GTEX-NPJ7-0011-R10A-SM-HAKXW   
4                                GENIE.GENIE-NKI-U4QV   
..                                                ...   
33                          GENIE.GENIE-GRCC-18fce8dd   
34                          GENIE.GENIE-GRCC-774ad24a   
35                          GENIE.GENIE-GRCC-22355f27   
36                          GENIE.GENIE-GRCC-4ac049a8   
37                          GENIE.GENIE-GRCC-99a7f076   

                                   subject_identifier       species  \
0   [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
1   [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
2   [{'system': 'PDC', 'field_name': 'Case.case_id...  Homo sapiens   
3   [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
4   [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
..                                                ...           ...   
33  [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
34  [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
35  [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
36  [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   
37  [{'system': 'GDC', 'field_name': 'case.submitt...  Homo sapiens   

             sex                    race               ethnicity  \
0           male                 Unknown                 Unknown   
1   not reported            not reported            not reported   
2         Female                   White                    None   
3         female                   white            not reported   
4         female                 Unknown                 Unknown   
..           ...                     ...                     ...   
33        female  not allowed to collect  not allowed to collect   
34        female  not allowed to collect  not allowed to collect   
35          male  not allowed to collect  not allowed to collect   
36          male  not allowed to collect  not allowed to collect   
37          male  not allowed to collect  not allowed to collect   

   days_to_birth                  subject_associated_project  vital_status  \
0       -24837.0                                 [GENIE-NKI]  Not Reported   
1            NaN                                  [TCGA-GBM]  Not Reported   
2            NaN           [PDC000204, PDC000205, PDC000245]          Dead   
3            NaN  [CPTAC-3, PDC000204, PDC000205, PDC000245]          Dead   
4       -27759.0                                 [GENIE-NKI]  Not Reported   
..           ...                                         ...           ...   
33      -26663.0                                [GENIE-GRCC]  Not Reported   
34      -11688.0                                [GENIE-GRCC]  Not Reported   
35      -23741.0                                [GENIE-GRCC]  Not Reported   
36      -20454.0                                [GENIE-GRCC]  Not Reported   
37      -17897.0                                [GENIE-GRCC]  Not Reported   

   days_to_death      cause_of_death  
0            NaN                None  
1            NaN                None  
2            NaN  Not Cancer Related  
3            0.0                None  
4            NaN                None  
..           ...                 ...  
33          None                None  
34          None                None  
35          None                None  
36          None                None  
37          None                None  

[2438 rows x 11 columns]

## Results to a list

If you want results in a list, you'll need to use the paginator. Saving to a list works similarly to the dataframe call. The differences are:

- initiate a list not a DataFrame
- change `to_list=True`
- change the concat index to `extend()`

In [8]:
mylist = []
for i in myquery.subject.run().paginator(to_list=True):
    mylist.extend(i)

Getting results from database

Total execution time: 0
                            min 3.667 sec 3667 ms

This gives back the correct number of results:

In [9]:
len(mylist)

2438

And we can preview the first result to see that it has the same values:

In [10]:
mylist[0:1]

[{'subject_id': 'GENIE.GENIE-NKI-8IN3',
  'subject_identifier': [{'system': 'GDC',
    'field_name': 'case.submitter_id',
    'value': 'GENIE-NKI-8IN3'}],
  'species': 'Homo sapiens',
  'sex': 'male',
  'race': 'Unknown',
  'ethnicity': 'Unknown',
  'days_to_birth': -24837,
  'subject_associated_project': ['GENIE-NKI'],
  'vital_status': 'Not Reported',
  'days_to_death': None,
  'cause_of_death': None}]